# 1 Knapsack Problem

The knapsack problem is one of the most trivial NP-hard problems. We can encounter many of its variants in the literature, which generally have various solution requirements for the algorithm. The formulation always contains criterium for the overall price, which means it is an optimization problem.

## 1.1 Author of Report

* **Name:** Tomáš Patro
* **Username:** patrotom
* **Email:** patrotom@fit.cvut.cz

## 1.2 Base of Problem

We are given

* integer $n$ (number of items)
* integer $M$ (capacity of knapsack)
* finite set $V = \{v_1, v_2, \dots, v_n\}$ (weights of items)
* finite set $C = \{c_1, c_2, \dots, c_n\}$ (prices of items)

## 1.3 Construction 0/1 Problem Form

The most well-known is the construction form. If we talk about the “knapsack problem” without a further description, we usually mean this version:

Construct the set $X = \{x_1, x_2, \dots, x_n\}$ where each $x_i$ is either $0$ or $1$, so $$v_1x_1 + v_2x_2 + \dots + v_nx_n \leq M \text{(knapsack is not overloaded)}$$ and expression $$c_1x_1 + c_2x_2 + \dots + c_nx_n$$ gains maximal value for all given sets (value of the items in the knapsack is maximal).

<br>

(“B201-NI-KOP: Problém Batohu (Knapsack Problem)” 2020)

# 2 Solution approaches

In the following section, we will describe approaches (algorithms) to solve the given problem:

* Brute force
* Branch and bound (B&B)
* Greedy heuristic
* Redux heuristic
* Dynamic programming (decomposition by weight)

Brute force, B&B, and dynamic programming algorithms find the optimal price. Greedy, and redux algorithms may not find the optimal price, and we track the relative error of the computation. We compare the found price with the optimal price computed by the B&B algorithm.

## 2.1 Brute Force

In the brute force approach, we search the whole state space to find the optimal price and configuration.

Let us look at the code snippet of the algorithm:

``` python
def brute_force(conf, i, weight, price):
    if i == instance.size:
        if price >= solution.price and weight <= instance.capacity:
            solution.price = price
            solution.conf = conf
        return

    new_weight = instance.items[i].weight + weight
    new_price = instance.items[i].price + price

    conf[i] = 1
    brute_force(conf, i + 1, new_weight, new_price)

    conf[i] = 0
    brute_force(conf, i + 1, weight, price)
```

The algorithm starts by comparing the current depth of the recursion `i` with the size of the instance. We use to comparison to determine if we have already solved one of the branches of recursion. If the `price` is greater or equal to the current optimal price and the `weight` is less than or equal to the knapsack capacity, we update the `solution` object with the new values – `price` and vector configuration `conf`.

Next, we compute a new weight by adding the current weight to the particular items' weight. To cover the whole state space, we have to call the recursion with the current bit set to either $0$ or $1$. We also have to recalculate the price and weight. We later send them the recursion if the bit is set to $1$ – it means that the item is added to the knapsack. In this way, we construct the whole state space and find the optimal solution.

## 2.2 Branch and Bound (B&B)

The beginning of the B&B algorithm is the same as in the brute force algorithm. The only difference is in the condition we use for the cutting of the recursion calls (state space). Let us take a look at the difference in the algorithm:

``` python
def branch_and_bound(conf, i, weight, price):
    if i == instance.size:
        if price >= solution.price:
            solution.price = price
            solution.conf = conf
        return
    
    new_weight = instance.items[i].weight + weight
    new_price = instance.items[i].price + price
    upper_bound = price + instance.prices_sum(i=i)

    conf[i] = 1
    if (new_weight <= instance.capacity) and (upper_bound > solution.price):
        branch_and_bound(conf, i + 1, new_weight, new_price)

    conf[i] = 0
    branch_and_bound(conf, i + 1, weight, price)
```

We add the condition by checking if the current `price` added to the sum of prices of the unvisited items (on the index `i` and greater) is greater than the current optimal price. We also check if the `new_weight` does not overload the knapsack. We can cut the recursion branch if we do not meet this condition. This way, we cut the branches of the state space, which do not lead us to the optimal price and valid solution.

## 2.3 Greedy Heuristic

We implement a simple heuristic using the relation between the price and weight. Let us take a look at the algorithm:

``` python
def greedy():
    instance.sort_items()
    capacity = instance.capacity
    price = 0

    for item in instance.items:
        if capacity >= item.weight:
            price += item.price
            capacity -= item.weight
            solution.conf[item.index] = 1
    
    solution.price = price
```

We start by sorting the items in descending order by $price/weight$ ratio. Then we iterate the items in this order. We check if the current capacity is greater or equal to the current items' weight in each iteration. If we meet this condition, we add the items' price to the overall price and lowers the capacity by the items' weight. We also update the configuration because we know that the item will be present in the result.

## 2.4 Redux Heuristic

Since we can encounter a situation where, for example, the last item alone would give us the optimal solution, we enhance the greedy heuristic in the following fashion:

``` python
def redux():
    greedy = Greedy()
    greedy.solve()
    highest_price, index = find_highest_price()

    if greedy.solution.price > highest_price:
        solution.price = greedy.solution.price
        solution.conf = greedy.solution.conf
    else:
        solution.price = highest_price
        solution.conf[index] = 1

def find_highest_price():
    highest_price = 0
    index = -1
    for item in instance.items:
        if item.price > highest_price and item.weight <= instance.capacity:
            highest_price = item.price
            index = item.index
    
    return (highest_price, index)
```

First, we calculate the price using the greedy algorithm. Next, we find the highest price of an item that does not overload the knapsack. Then, we compare the price found by the greedy algorithm with the highest price – the solution depends on the price, which is higher.

## 2.5 Dynamic Programming (Weight Decomposition)

The subproblems are re-evaluated, and the problem has overlapping subproblems property. We can take advantage of the dynamic programming by constructing a temporary 2D array `table` in a bottom-up manner:

``` python
def dynamic_by_weight():
    capacity = instance.capacity
    size = instance.size
    table = [[0 for x in range(capacity + 1)] for x in range(size + 1)]

    for i in range(size + 1):
        for w in range(capacity + 1):
            weight = instance.items[i - 1].weight
            price = instance.items[i - 1].price

            if i == 0 or w == 0:
                table[i][w] = 0
            elif weight <= w:
                table[i][w] = max(
                    price + table[i - 1][w - weight],
                    table[i - 1][w]
                )
            else:
                table[i][w] = table[i - 1][w]

    solution.price = table[size][capacity]
    solution.conf = construct_conf(capacity)

def construct_conf():
    conf = []

    for i in range(instance.size, 0, -1):
        if table[i][capacity] == table[i - 1][capacity]:
            conf.append(0)
        else:
            conf.append(1)
            capacity -= inst.items[i - 1].weight

    conf.reverse()

    return conf
```

In this solution, we take an approach where we compare the situation where the $n$th item is in the solution and when it is not. When we compare these two situations, we determine whether the item should or should not be in the solution. We repeat this process unless we encounter trivial subproblems. This approach leads us to an optimal solution. Afterward, we iterate over the table in a backward fashion to reconstruct the optimal solution's configuration.

# 3 Experimental Evaluation of Algorithms Quality

There is a lot of different algorithms for solving the knapsack problem. However, we do not know too much about their behavior. If we want to gather information about their behavior, we have to evaluate them experimentally.

We observe two important aspects:

1. quality of the solution
2. execution time

We observe these aspects in relation to different parameters of an instance. If we put these two aspects into relation, we can figure out which algorithm is the most suitable based on the combination of the requirements on these aspects. We also observe the stability of the algorithm given a different order of the items – robustness.

We use a prepared generator of the instances *kg2*, which can be parametrized, and thus, we can observe how the algorithm behaves under different conditions. We relate to the different behavior of an algorithm as to the sensitivity of the algorithm. We assume that the different parameters do not relate to each other – we set the parameters to the constant values and always change only one parameter.

**Data analysis:**

We analyze the data using the *Python* language and its mathematical modules. The graphs and the whole report are rendered in the *Jupyter Notebook* – that's why we also add the code snippets in this report, which process data and generate the graphs.

**Execution time measuring:**

We measured the execution time for each run of an algorithm and saved the data to the prepared object. All execution times in this report were measured using the Python `timeit` module, which measures CPU time. The times you see in the following graphs are represented in **microseconds \[μs\]**.

Computer configuration:

* **CPU:** Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz
* **OS:** Linux Mint 19.2
* **Linux kernel:** 5.4.0-48-generic